In [1]:
import os
from pathlib import Path

def root_dir(marker="EXPLib"):
    cwd = Path.cwd()
    # if cwd already contains the marker
    if (cwd / marker).is_dir():
        return cwd
    # otherwise walk up the parents
    for p in cwd.parents:
        if (p / marker).is_dir():
            return p
    raise FileNotFoundError(f"Root marker '{marker}' not found")

os.chdir(root_dir())
#print(os.getcwd())

In [2]:
import torch
import torchvision
from EXPLib.xai_library.metric_extractors.structural_coverage import NeuronCoverage, TopKNeuronCoverage, NeuronBoundaryCoverage
from EXPLib.xai_library.model_explainers.model_utils.toymodel_utils import load_and_configure_model, preprocess_image

In [3]:
weight_path = './EXPLib/dl_component/CNN/Object_Detectors/toy_model_v1'
img_path = './EXPLib/datasets/Camera/toy_model_v1_inference/dev/images'
image_file='rgb_0000079.png'
model_name = 'ssdlite320_mobilenet_v3_large'
weight_file = 'aikonet_toymodelv1_ssdlite.pth'
num_classes = 2
im_size = 320

device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [4]:
toymodel = load_and_configure_model(model_name, weight_path, weight_file, num_classes, device)
image = preprocess_image(img_path, image_file, device)

In [ ]:
toymodel.eval()
input_tensor = image

nc = NeuronCoverage(toymodel)
tknc = TopKNeuronCoverage(toymodel, top_k=500)
nbc = NeuronBoundaryCoverage(toymodel)

neuron_cov = nc.compute_coverage(input_tensor)
topk_cov = tknc.compute_coverage(input_tensor)
boundary_cov = nbc.compute_boundary_coverage(input_tensor, input_tensor + 0.01)

print(f"Neuron Coverage: {neuron_cov:.4f}%")
print(f"Top-K Neuron Coverage: {topk_cov:.4f}%")
print(f"Boundary Coverage: {boundary_cov:.4f}%")

nc.clear_hooks()
tknc.clear_hooks()
nbc.clear_hooks()

Neuron Coverage: 48.7696%
Top-K Neuron Coverage: 0.0053%
Boundary Coverage: 95.9771%
